In [52]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import nltk
import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [47]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv("D:\Data for python\datasets\cyber bullying tweets\cyberbullying_tweets.csv") 

In [5]:
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [7]:
df.cyberbullying_type.unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [9]:
bullying_type = { "not_cyberbullying":0, "gender":1, "religion":2, "age":3, "ethnicity":4, "other_cyberbullying":5}

In [10]:
df.cyberbullying_type = df.cyberbullying_type.apply(lambda x: bullying_type[x])

In [17]:
df.tweet_text = df.tweet_text.apply(lambda x: x.lower())

In [18]:
df

,tweet_text,cyberbullying_type
0,"in other words #katandandre, your food was cra...",0
1,why is #aussietv so white? #mkr #theblock #ima...,0
2,@xochitlsuckkks a classy whore? or more red ve...,0
3,"@jason_gio meh. :p thanks for the heads up, b...",0
4,@rudhoeenglish this is an isis account pretend...,0
...,...,...
47687,"black ppl aren't expected to do anything, depe...",4
47688,turner did not withhold his disappointment. tu...,4
47689,i swear to god. this dumb nigger bitch. i have...,4
47690,yea fuck you rt @therealexel: if youre a nigge...,4


preparing GLOVE word embeddings

In [51]:
EMBED_DIM = 100

glove_path = fr"D:\Data for python\datasets\glove.twitter.27B\glove.twitter.27B.{EMBED_DIM}d.txt"

def dict_from_embeddings(filename):
    embeddings = {}
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            line = line.split(' ')

            try:
                embeddings[line[0]] = np.array(line[1:], dtype=float)
            except:
                continue
    return embeddings

In [13]:
embedding_dict = dict_from_embeddings(glove_path)

In [14]:
len(embedding_dict)

1193515

In [28]:
SPLIT_CUT =  38000   #

x = df.drop("cyberbullying_type",axis=1)
y = df.cyberbullying_type

x_train_full, x_test, y_train_full, y_test =  train_test_split(x, y, test_size=0.1)          
x_train, x_valid =  x_train_full[:SPLIT_CUT], x_train_full[SPLIT_CUT:]   
y_train, y_valid =  y_train_full[:SPLIT_CUT], y_train_full[SPLIT_CUT:]

presence of special characters.
what to do with them?

In [46]:
# hashtags
ht = df[df.tweet_text.str.contains("#")].shape[0] / df.shape[0]
mentions = df[df.tweet_text.str.contains("@")].shape[0] / df.shape[0]

print(f"hashtags: {round(ht * 100,2 )}%")
print(f"mentions: {round(mentions *100,2)} %")

hashtags: 14.7%
mentions: 38.03 %


what if I swap @ to "mentions " and # to "about"

In [50]:
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_tweets(data, embedding_dict):
    processed_tweets = []
    for tweet in data:
        # Remove URLs
        tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)

        # Replace user mentions and hashtags with special tokens
        tweet = re.sub(r'\@\w+', 'mentions ', tweet)
        tweet = re.sub(r'\#\w+', 'about', tweet)

        # Convert to lowercase
        tweet = tweet.lower()

        # Tokenize using the NLTK TweetTokenizer
        tokenizer = nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        tokens = tokenizer.tokenize(tweet)

        # Remove stopwords and lemmatize
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords]

        # Convert tokens to embeddings
        embedding_vectors = []
        for token in lemmatized_tokens:
            if token in embedding_dict:
                embedding_vectors.append(embedding_dict[token])

        # Pad embeddings with zeros to ensure a fixed sequence length
        embedding_vectors = np.pad(embedding_vectors, pad_width=((0, EMBED_DIM - len(embedding_vectors)), (0, 0)), mode='constant')

        processed_tweets.append(embedding_vectors)

    return np.array(processed_tweets)